In [1]:
import pandas as pd
import datetime as dt

In [2]:
f = lambda s: dt.datetime.strptime(s,'%d-%m-%Y %H:%M:%S')
df_bn5 = pd.read_csv("data/historical/BankNiftyFut-Sep-5min-2.csv", sep="\t", parse_dates=['Date Time'], date_parser=f, encoding="utf_16_LE") 

#### Remove last empty column and set date time as index

In [3]:
df_bn5 = df_bn5[df_bn5.columns[:-1]]
df_bn5.set_index('Date Time', inplace=True)

#### Apply rolling mean columns for data

In [4]:
df_bn5['SMA6'] = df_bn5['Close'].rolling(6).mean()
df_bn5['SMA21'] = df_bn5['Close'].rolling(21).mean()

## Input the evaluation date below

In [39]:
eval_date = "2020-08-28"

#### Extract a separate dataframe for the evaluation date

In [40]:
today_df_bn5 = df_bn5[eval_date]

In [41]:
today_df_bn5.head(5)

,Open,Low,High,Close,Volume,SMA6,SMA21
Date Time,,,,,,,
2020-08-28 09:15:00,23679.90,23601.50,23795.00,23795.00,335203,23631.233333,23625.642857
2020-08-28 09:20:00,23797.00,23797.00,23828.55,23822.55,203275,23670.175000,23636.233333
2020-08-28 09:25:00,23823.55,23810.00,23850.00,23847.00,140175,23712.175000,23646.804762
2020-08-28 09:30:00,23847.00,23844.40,23885.00,23860.90,169025,23757.658333,23658.338095
2020-08-28 09:35:00,23857.75,23845.05,23886.85,23882.45,95700,23802.491667,23670.930952


### Strategy function

In [42]:
def backtest_Intraday_5min_strategy(df):
    SELL = False
    BUY = False
    SELL_PRICE = 0
    BUY_PRICE = 0
    GAIN_POINTS = 0
    SELL_STOPLOSS = 0
    BUY_STOPLOSS = 0
    STOPLOSS = 100
    SMA6_dead_band = 50
    Total_Signals = 0
    Total_Sell_Signal = 0
    Total_Buy_Signal = 0
    start_time = dt.datetime.strptime("10:00", "%H:%M").time()
    close_time = dt.datetime.strptime("15:15", "%H:%M").time()
    entry_close_time = dt.datetime.strptime("15:00", "%H:%M").time()
    print("Time\t\t\t", "Close\t\t", "SMA6\t\t", "SMA21\t\t", "SIGNAL\t", "TYPE\t\t",  "Points\t",)
    
    for row in df.itertuples():
        SMA6_upper_band = row.SMA6 + SMA6_dead_band
        SMA6_lower_band = row.SMA6 - SMA6_dead_band
        
        # SELL STOPLOSS
        if row.Index.time() >= start_time and row.Index.time() <= close_time: 
            points = 0
            if SELL and row.Close > SELL_STOPLOSS:
                SELL = False
                BUY = False
                Total_Signals += 1
                Total_Buy_Signal += 1
                points = SELL_PRICE - row.Close
                GAIN_POINTS = GAIN_POINTS + points
                SELL_PRICE = 0
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "SELL STOPLOSS\t", round(points,2))
            
            # BUY STOPLOSS
            points = 0
            if BUY and row.Close < BUY_STOPLOSS:
                points = row.Close - BUY_PRICE
                GAIN_POINTS = GAIN_POINTS + points
                BUY_PRICE = 0
                SELL = False
                BUY = False
                Total_Signals += 1
                Total_Sell_Signal += 1
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "BUY STOPLOSS\t", round(points,2))
            
            # SELL ENTRY
            points = 0
            if row.Close < SMA6_lower_band and \
                    row.Index.time() < entry_close_time and \
                    SELL is False:
                SELL_PRICE = row.Close
                SELL_STOPLOSS = SELL_PRICE + STOPLOSS
                if BUY_PRICE > 0:
                    points = SELL_PRICE - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                SELL = True
                BUY = False
                Total_Sell_Signal += 1
                Total_Signals += 1
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "SELL ENTRY\t", round(points,2))
             
            # BUY ENTRY
            points = 0
            if row.Close > SMA6_upper_band and \
                    row.Index.time() < entry_close_time and \
                    BUY is False:
                BUY_PRICE = row.Close
                BUY_STOPLOSS = BUY_PRICE - STOPLOSS
                if SELL_PRICE > 0:
                    points = SELL_PRICE - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                BUY = True
                SELL = False
                Total_Buy_Signal += 1
                Total_Signals += 1
                print(row.Index, "\t", row.Close,  "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "BUY ENTRY\t", round(points,2))
            
            # DAY Close
            points = 0    
            if row.Index.time() >= close_time:
                if SELL:
                    SELL = False
                    BUY = False
                    Total_Signals += 1
                    Total_Buy_Signal += 1
                    points = SELL_PRICE - row.Close
                    GAIN_POINTS = GAIN_POINTS + points
                    SELL_PRICE = 0
                    print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "BUY DAYCLOSE\t", round(points,2))

                if BUY:
                    SELL = False
                    BUY = False
                    Total_Signals += 1
                    Total_Buy_Signal += 1
                    points = row.Close - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                    SELL_PRICE = 0
                    print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "SELL DAYCLOSE\t", round(points,2))
                    
                    
    
    print("\n\nTotal Signals:\t\t", Total_Signals)
    print("Total Sell Signals:\t", Total_Buy_Signal)
    print("Total Buy Signals:\t", Total_Sell_Signal)
    print("Total Points Gained:\t", round(GAIN_POINTS,2))

In [43]:
backtest_Intraday_5min_strategy(today_df_bn5)

Time			 Close		 SMA6		 SMA21		 SIGNAL	 TYPE		 Points	
2020-08-28 10:30:00 	 24105.0 	 23950.96 	 23827.08 	 BUY	 BUY ENTRY	 0
2020-08-28 12:20:00 	 24303.05 	 24361.45 	 24267.7 	 SELL	 SELL ENTRY	 198.05
2020-08-28 13:35:00 	 24324.9 	 24271.33 	 24289.94 	 BUY	 BUY ENTRY	 -21.85
2020-08-28 15:15:00 	 24549.35 	 24478.71 	 24401.21 	 SELL	 SELL DAYCLOSE	 224.45


Total Signals:		 4
Total Sell Signals:	 3
Total Buy Signals:	 1
Total Points Gained:	 400.65
